# Fine Tuning GPT-3.5-Turbo

In this notebook, we walk through an example of fine-tuning gpt-3.5-turbo.

Specifically, we attempt to distill GPT-4's knowledge, by generating training data with GPT-4 to then fine-tune GPT-3.5.

All training data is generated using two different sections of our index data, creating both a training and evalution set.

Evaluation is done using the `ragas` library, which we will detail later on.

In [ ]:
!pip install llama-index pypdf sentence-transformers ragas

In [ ]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "EMPTY"
openai.api_key = os.environ["OPENAI_API_KEY"]
openai.base_path = "http://localhost:1234/v1"
openai.api_base = "http://localhost:1234/v1"

## Data Setup

Here, we first load the text file that we will use to generate training data.

In [ ]:
file = 'theme-notes.txt'

The next step is generating a training and eval dataset.

We will generate 40 questions on different sections of the text file we downloaded.

We can use GPT-3.5 on the eval questions to get our baseline performance.

Then, we will use GPT-4 on the train questions to generate our training data. The training data will be collected with out `OpenAIFineTuningHandler`.

This step is entirely optional if you don't want to spend the time/tokens -- the eval and training questions are also provided in this folder, as well as the training data!

### Train Generation

In [ ]:
from llama_index import SimpleDirectoryReader, ServiceContext, Document
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator
import random
from llama_index.text_splitter import SentenceSplitter
import tiktoken
from llama_index.utils import globals_helper
from langchain.text_splitter import (
    NLTKTextSplitter,
    SpacyTextSplitter,
    RecursiveCharacterTextSplitter,
)

We need to split a given text file up into separate documents so that we can create a training and evaluation set.

In [ ]:
# replace "data" with the FOLDER containing your data
document = SimpleDirectoryReader("data").load_data()[0]
text_splitter_default = SentenceSplitter(
  separator=" ",
  chunk_size=1024,
  chunk_overlap=20,
  paragraph_separator="\n\n\n\n",
  secondary_chunking_regex="[^,.;。]+[,.;。]?",
  tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)
text_chunks = text_splitter_default.split_text(document.text)
doc_chunks = [Document(text=t) for t in text_chunks]
tokenizer = globals_helper.tokenizer
with open("splitting_1.txt", "w") as f:
    for idx, doc in enumerate(doc_chunks):
        f.write(
            "\n-------\n\n{}. Size: {} tokens\n".format(idx, len(tokenizer(doc.text)))
            + doc.text
        )

In [ ]:
for idx, doc in enumerate(doc_chunks):
  with open(f'/content/documents/{idx}.txt', 'w') as file:
    file.write(doc.text)

In [ ]:
documents = SimpleDirectoryReader(
    input_dir= '/content/documents'
).load_data()

# Shuffle the documents
random.seed(42)
random.shuffle(documents)

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3)
)

In [ ]:
question_gen_query = (
    "You are a Teacher/ Professor. Your task is to setup "
    "a quiz/examination. Using the provided context from a "
    "series of themes found in biblical passages, formulate "
    "a single question that captures an important fact from the "
    "context. Restrict the question to the context information provided."
)

dataset_generator = DatasetGenerator.from_documents(
    documents[:50],
    question_gen_query=question_gen_query,
    service_context=gpt_35_context,
)

In [ ]:
# NOTE: this may take some time. Go grab a coffee! (or tea if you have taste buds)
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

In [ ]:
questions[:5]

In [ ]:
with open("train_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

### Eval Generation

Now, lets generate questions on a completely different set of documents, in order to create our eval dataset.

In [ ]:
dataset_generator = DatasetGenerator.from_documents(
    documents[
        50:
    ],  # since we generated ~1 question for 40 documents, we can skip the first 40
    question_gen_query=question_gen_query,
    service_context=gpt_35_context,
)

In [ ]:
# NOTE: this may take some time. Go grab a coffee! (or tea if you have tastebuds)
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

In [ ]:
with open("eval_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

## Initial Eval with GPT-3.5-Turbo Query Engine

For this eval, we will be using the [`ragas` evaluation library](https://github.com/explodinggradients/ragas).

Ragas has a ton of evaluation metrics for RAG pipelines, and you can read about them [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md).

For this notebook, we will be using the following two metrics

- `answer_relevancy` - This measures how relevant is the generated answer to the prompt. If the generated answer is incomplete or contains redundant information the score will be low. This is quantified by working out the chance of an LLM generating the given question using the generated answer. Values range (0,1), higher the better.
- `faithfulness` - This measures the factual consistency of the generated answer against the given context. This is done using a multi step paradigm that includes creation of statements from the generated answer followed by verifying each of these statements against the context. The answer is scaled to (0,1) range. Higher the better.

In [ ]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [ ]:
from llama_index import VectorStoreIndex

# limit the context window to 2048 tokens so that refine is used
gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3), context_window=2048
)

index = VectorStoreIndex.from_documents(documents, service_context=gpt_35_context)

query_engine = index.as_query_engine(similarity_top_k=2)

In [ ]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)

## GPT-4 to Collect Training Data

Here, we use GPT-4 and the `OpenAIFineTuningHandler` to collect data that we want to train on.

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)

In [ ]:
questions = []
with open("train_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, service_context=gpt_4_context)

query_engine = index.as_query_engine(similarity_top_k=2)

In [ ]:
for question in questions:
    response = query_engine.query(question)

## Create Fine-Tuning Data

Fine-Tuning data must be written as a list of messages in a `.jsonl` file. Using the finetuning-handler, we can easily write the messages to a `.jsonl` file.

In [ ]:
finetuning_handler.save_finetuning_events("finetuning_events.jsonl")

## Launch Fine-Tuning Job

In [ ]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

In [ ]:
from datasets import load_dataset
from random import randrange

dataset = load_dataset("json", data_files="/content/finetuning_events.jsonl",download_mode="force_redownload", split="train")

print(f"dataset size: {len(dataset)}")
print(dataset)

In [ ]:
def format_sample(sample):
  return f'''
  ## Instruction:
  {sample['messages'][0]['content']}

  ## Context:
  {sample['messages'][1]['content']}

  ## Answer:
  {sample['messages'][2]['content']}
  '''

In [ ]:
for prompt in dataset:
    print(format_sample(prompt))

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False

model_id = "NousResearch/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map="auto")
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="llama-7-int4-dolly",
    num_train_epochs=3,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    fp16=False,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False,  # disable tqdm since with packing values are in correct
)

model = get_peft_model(model, peft_config)

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_sample,
    args=args,
)

In [ ]:
trainer.train()
trainer.save_model()

## Evaluation

After some time, your model will be done training!

The next step is running our fine-tuned model on our eval dataset again to measure any performance increase.

In [ ]:
ft_model_name = "ft:gpt-3.5-turbo-0613:..."

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager


ft_context = ServiceContext.from_defaults(
    llm=OpenAI(model=ft_model_name, temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [ ]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, service_context=ft_context)

query_engine = index.as_query_engine(similarity_top_k=2)

In [ ]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)

## Exploring Differences

Let's quickly compare the differences in responses, to demonstrate that fine tuning did indeed change something.

In [ ]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

In [ ]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [ ]:
print(questions[12])

### Original

In [ ]:
from llama_index.response.notebook_utils import display_response
from llama_index import ServiceContext
from llama_index.llms import OpenAI


gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [ ]:
query_engine = index.as_query_engine(service_context=gpt_35_context)

response = query_engine.query(questions[12])

display_response(response)

### Fine-Tuned

In [ ]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI


ft_context = ServiceContext.from_defaults(
    llm=OpenAI(model=ft_model_name, temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [ ]:
query_engine = index.as_query_engine(service_context=ft_context)

response = query_engine.query(questions[12])

display_response(response)

As we can see, the fine-tuned model provides a more thorough response! This lines up with the increased faithfullness score from ragas, since the answer is more representative of the retrieved context.

## Conclusion

So, in conclusion, finetuning with only ~61 questions actually helped improve our eval scores!

**answer_relevancy: 0.9778 -> 0.9758**

The answer relenvancy appears to be basically unchanged, between models.

**faithfulness: 0.7638 -> 0.8088**

The faithfulness appears to have been improved! This mains the anwers given better fuffil the original question that was asked.